In [1]:

from common_functions import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


In [2]:
import numpy as np
import pandas as pd
import scipy as sp
# import AgglomerativeClustering from sklearn
from sklearn.cluster import AgglomerativeClustering
from math import sqrt


In [3]:
# Creating MxN matrix where M is the number of users and N is the number of books
# an Example of the representation of the matrix is shown below
#       Items  I1        I2        I3      I4      I5
# Users
# U1           1.0       4.0       5.0     3.0     2.0
# U2           2.0       3.0       4.0     5.0     1.0
# U3           0.0       4.0       3.0     0.0     1.0
# U4           1.0       2.0       3.0     4.0     5.0
# U5           5.0       4.0       3.0     2.0     1.0

read_data = read_data(RATINGS_DF_PATH)
data_shrinking = data_shrinking(read_data)
ratings_matrix = create_ratings_matrix(data_shrinking)
ratings_matrix_centered = mean_centered_rating_matrix(ratings_matrix)
ratings_matrix_centered.head()






,user_id,book_id,review_id,review_text,rating
501,01ec1a320ffded6b2dd47833f2c8e4fb,18812405,5a81d68f7eaa7aaad499dfd0a6c97824,Well. I was prepared to love this book. Even t...,4
517,01ec1a320ffded6b2dd47833f2c8e4fb,7937843,cf94492e1449528b471fb6b5a9f82d74,Absolute unforgettable story of the power of a...,5
767,01ec1a320ffded6b2dd47833f2c8e4fb,24920901,b0a118c65aa9d97a62c2c02157bf5af3,"4.5 - 5 Sexy - Sweet Stars!! \n Hello, Elle Ke...",5
785,01ec1a320ffded6b2dd47833f2c8e4fb,27362503,849d3df919cf08a01c3d24ed9732ca63,"""Someone once told me that there is no such th...",5
843,01ec1a320ffded6b2dd47833f2c8e4fb,24378015,847c8dc0abbd022367b83953af51ad43,What do you get when you mix Colleen Hoover an...,5


In [48]:
mean_centered_matrix = rating_matrix.sub(rating_matrix.mean(axis=1), axis=0)
print(mean_centered_matrix.shape)
mean_centered_matrix.head()


(899, 347)


book_id,1,2,3,5,6,11,865,960,968,1232,...,27161156,27362503,27833670,27883214,28187230,28260587,29056083,29069989,30555488,32075671
user_id,,,,,,,,,,,,,,,,,,,,,
004d5e96c8a318aeb006af50f8cc949c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.646465,1.353535,-0.646465,-0.646465,-1.646465,1.353535,1.353535,NaN,-1.646465,1.353535
00660d8ef237d83e8b412c3d6d85b860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0076463c51f453d754cdd95dfb579dc5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
009ed6643b45c30f1c759284a72a9365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.136364,NaN,NaN,NaN,NaN,0.136364,NaN,NaN,NaN
00ce07379fb4a962964dcfde4e146a84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## User-based collaborative filtering

In [49]:
user_pearson_similarity_matrix = rating_matrix.copy()

# use corr function to calculate pearson correlation between users (columns)
user_pearson_similarity_matrix = user_pearson_similarity_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)
user_pearson_similarity_matrix = user_pearson_similarity_matrix.T.corr(method="pearson")





In [50]:

def similar_users(userID: str, top_percent: int, threshold: float = 0.2):
    index = rating_matrix.index.get_loc(userID)
    row_rating = user_pearson_similarity_matrix.iloc[index]
    row_rating.sort_values(ascending=False, inplace=True)
    top_users = round(len(user_pearson_similarity_matrix) * top_percent)
    user_list = list(row_rating.index[1:top_users+1])
    # remove users with similarity less than threshold
    user_list = [user for user in user_list if row_rating[user] > threshold]
    return user_list

def find_not_rated_books(userID: str):
    not_rated_books = rating_matrix.loc[userID][rating_matrix.loc[userID].isnull()].index
    return list(not_rated_books)


# this function predicts the rating of the missing values in the rating matrix for a given user given a list of similar users
def predict_rating_user(userID: str , userIDs: list):
    # get the not rated books by the user
    not_rated_books = find_not_rated_books(userID)
    # dictionary to store the predicted rating for each book
    prediction_dict = {}
    for book in not_rated_books:
        numerator = 0
        denominator = 0
        for user in userIDs:
            if not np.isnan(rating_matrix.loc[user][book]):
                numerator += user_pearson_similarity_matrix.loc[userID][user] * mean_centered_matrix.loc[user][book]
                denominator += user_pearson_similarity_matrix.loc[userID][user]
        if denominator != 0:
            prediction_dict[book] = numerator / denominator + rating_matrix.loc[userID].mean()
    
    # sort the dictionary by the predicted rating in descending order
    prediction_dict = {i: v for i, v in sorted(prediction_dict.items(), key=lambda item: item[1], reverse=True)}
    return prediction_dict



In [51]:
top_users_percent = 0.3
mylist = similar_users("004d5e96c8a318aeb006af50f8cc949c", top_users_percent)
user_based_prediction = predict_rating_user("004d5e96c8a318aeb006af50f8cc949c", mylist)
print("recommended books for user are: (Item: Rating Prediction)")
print(user_based_prediction)



recommended books for user are: (Item: Rating Prediction)
{18293427: 4.8878439568094745, 6314763: 3.887843956809474, 17347389: 3.887843956809474, 17378508: 3.887843956809474, 17378527: 3.887843956809474, 21996: 2.887843956809474, 8664353: 2.887843956809474, 15797938: 2.887843956809474, 18710190: 2.887843956809474, 22489107: 2.887843956809474, 24612118: 2.887843956809474}


## Item-based collaborative filtering

In [52]:
item_pearson_similarity_matrix = rating_matrix.copy()
# use corr function to calculate pearson correlation between users (columns)
item_pearson_similarity_matrix = item_pearson_similarity_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)
item_pearson_similarity_matrix = item_pearson_similarity_matrix.corr(method="pearson")

In [53]:
def similar_items(itemID: str, top_items_number: int = len(rating_matrix.columns), threshold: float = 0.2):
    index = rating_matrix.columns.get_loc(itemID)
    coloumn_rating = item_pearson_similarity_matrix.iloc[index]
    coloumn_rating.sort_values(ascending=False, inplace=True)
    top_items = min(len(item_pearson_similarity_matrix), top_items_number)
    item_list = list(coloumn_rating.index[1:top_items+1])
    # remove items with similarity less than threshold
    item_list = [item for item in item_list if coloumn_rating[item] > threshold]
    return item_list



def predict_rating_item(userID: str):
    not_rated_books = find_not_rated_books(userID)
    # dictionary to store the predicted rating for each book
    prediction_dict = {}
    top_items_number = round(sqrt(len(rating_matrix.columns)))

    for book in not_rated_books:
        similar_books = similar_items(book, top_items_number)
        # print("similar books for book {} are:".format(book))
        # print(similar_books)
        numerator = 0
        denominator = 0
        for item in similar_books:
            if not np.isnan(rating_matrix.loc[userID][item]):
                numerator += item_pearson_similarity_matrix.loc[book][item] * rating_matrix.loc[userID][item]
                denominator += item_pearson_similarity_matrix.loc[book][item]
                # print("denominator: {}".format(denominator))
        if denominator != 0:
            prediction_dict[book] = numerator / denominator
        # print("----------------------------------------")
    # sort the dictionary by the predicted rating in descending order
    prediction_dict = {i: v for i, v in sorted(prediction_dict.items(), key=lambda item: item[1], reverse=True)}
    return prediction_dict

#### Item-based collaborative filtering test

In [57]:
# case item searched, with no clue who the user is
# item_prediction = similar_items(968)
# print(item_prediction)

# case user logged in
item_based_prediction = predict_rating_item(user_ID_test)
print(item_based_prediction)



{693208: 3.745759459713191, 7613: 3.7429052617425675, 23766634: 3.6648957241103846, 7624: 3.6627200203883454, 4214: 3.6622433507904644, 13496: 3.5981065756656134, 24378015: 3.4974062469178313, 77203: 3.4955611056739553, 865: 3.4926005901330375, 2728527: 3.333965574711275, 136251: 3.3326859169941527, 7735333: 3.3326614848017626, 13372690: 3.3324143498458874, 11925514: 3.331274390053539, 6936382: 3.3312713526400803, 18143950: 3.3312296861194852, 13335037: 3.3312202891217333, 15990969: 3.33107204563236, 6644117: 3.3307498503316633, 1812457: 3.329840699021469, 5886881: 3.329323604330539, 10032672: 3.328918810840688, 18774964: 3.3285916978600136, 7244: 3.328567815271462, 16248068: 3.2517691202646457, 16081272: 3.2514403948805444, 6131164: 3.250410568726962, 7686667: 3.2500880121615916, 17788401: 3.249795643730048, 764347: 3.2494461689609286, 10025305: 3.2493691401639166, 18692431: 3.24921658382007, 11614718: 3.2488546590646075, 5310515: 3.248844078562135, 20639274: 3.2486930835561108, 29140

## Final predictions & statistics

In [59]:
user_mean = rating_matrix.loc[user_ID_test].mean()
print("user mean: {}".format(user_mean))

## calculating average prediction from item and user CF
average_prediction = {}
common_items = set(item_based_prediction.keys()).intersection(set(user_based_prediction.keys()))
for item in common_items:
    average_prediction[item] = (item_based_prediction[item] + user_based_prediction[item]) / 2

print(average_prediction)

# calculating the RMSE
def rmse(prediction_dict: dict, user_mean: float):
    rmse = 0
    for item in prediction_dict:
        rmse += (prediction_dict[item] - user_mean) ** 2
    rmse = sqrt(rmse / len(prediction_dict))
    return rmse


print("RMSE for user based CF: {}".format(rmse(user_based_prediction, user_mean)))
print("RMSE for item based CF: {}".format(rmse(item_based_prediction, user_mean)))
print("RMSE for average prediction: {}".format(rmse(average_prediction, user_mean)))




user mean: 3.6464646464646466
{8664353: 2.943921978404737, 6314763: 3.443921978404737, 21996: 2.778672837842172, 17378508: 3.319115269022083, 18710190: 2.943921978404737, 15797938: 2.9439219784047372, 18293427: 3.9439219784047372, 22489107: 2.77732264321156, 24612118: 2.9439219784047372, 17347389: 3.4439219784047372, 17378527: 3.321349343769989}
RMSE for user based CF: 0.689341621512197
RMSE for item based CF: 0.6711585280714137
RMSE for average prediction: 0.5928452772828862
